In [1]:
import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [2]:
index = faiss.read_index("vector_store/faiss_index.index")

In [3]:
with open("vector_store/documents.pkl", "rb") as f:
    documents = pickle.load(f)

with open("vector_store/metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
def retrieve_chunks(question, k=5):
    """Embed question and retrieve top-k relevant chunks."""
    question_embedding = model.encode([question])
    _, indices = index.search(np.array(question_embedding), k)
    results = [documents[i] for i in indices[0]]
    result_meta = [metadata[i] for i in indices[0]]
    return results, result_meta

In [5]:
def build_prompt(context_chunks, question):
    context = "\n---\n".join(context_chunks)
    prompt = f"""You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints.

Use only the information from the retrieved complaint excerpts below.

If the context does not contain the answer, say: 'I don't have enough information to answer that.'

Context:
{context}

Question: {question}

Answer:"""
    return prompt


In [6]:
from transformers import pipeline

generator = pipeline(
    "text2text-generation",
    model=r"C:\Users\bless\OneDrive\Desktop\week _6\crediTrust-rag-chatbot\models\flan-t5-base",  # local path
    tokenizer=r"C:\Users\bless\OneDrive\Desktop\week _6\crediTrust-rag-chatbot\models\flan-t5-base",
    device=-1
)


Device set to use cpu


In [16]:
def generate_answer(question):
    
    answer = "This is a sample answer to: " + question
    sources = ["doc1", "doc2"]
    metadata = {"source_type": "demo"}

    return answer, sources, metadata


In [17]:
question = "Why are customers unhappy with BNPL?"
answer, sources, metadata = generate_answer(question)
print(answer)


This is a sample answer to: Why are customers unhappy with BNPL?
